In [1]:
import os
from dotenv import load_dotenv
HOME = os.getenv('HOME')
load_dotenv(f'{HOME}/.zshrc')

from utils.db import ToricCY
from utils import format
db = ToricCY()

In [2]:
invols = db.find_invols({'H11': 3})

In [3]:
test_invols = []
for x in invols:
    if x['VOLFORMPARITY'] == -1:
        for y in x['OPLANES']:
            if y['ODIM'] == 7:
                test_invols.append(dict(x))

In [41]:
from utils.calc import get_srideal
ex = invols[20]
rescws = format.mat(ex['RESCWS'])
itensxd = format.mat(ex['ITENSXD'])
nverts = format.mat(ex['NVERTS'])
dresverts = format.mat(ex['DRESVERTS'])
k, n = dresverts.shape
triang = format.mat(ex['TRIANG'])
srideal = get_srideal(triang)
invol = format.invol(ex['INVOL'], k)

In [42]:
import numpy as np
from sage.geometry.polyhedron.constructor import Polyhedron
Delta = Polyhedron(vertices=nverts)
npoints = np.array(Delta.integral_points())

In [43]:
P_expon = (npoints @ dresverts.T) + 1

In [44]:
from sage.rings.integer_ring import ZZ
from sage.matrix.all import matrix
m = matrix(ZZ, P_expon - P_expon[:, invol.array_form]).row_space().matrix()

In [45]:
m

[ 1  0 -1  1  0 -1  0]

In [46]:
ex['OPLANES']

[{'OIDEAL': ['x1*x4-x3*x6'], 'ODIM': 7}]

In [47]:
P_weights = np.unique(P_expon @ rescws, axis=0)

In [48]:
P_weights

array([[4, 4, 4]])

In [49]:
from sage.rings.rational_field import QQ
R = QQ[tuple(f'x_{i}' for i in range(k))]
X = R.gens()
X_invol = [X[i] for i in invol.array_form]

In [50]:
srideal = [R.ideal([X[i] for i in x]) for x in srideal]

In [51]:
P_monoms = np.power(X, P_expon).prod(axis=1)
P = P_monoms.sum()

In [52]:
P_invol_monoms = np.power(X_invol, P_expon).prod(axis=1)
P_invol = P_invol_monoms.sum()

In [77]:
P_diff = P_monoms - P_invol_monoms
P_diff = P_diff[P_diff.nonzero()]
P_diff = R.ideal(*P_diff)

In [78]:
fixed_loci = P_diff.minimal_associated_primes()

In [24]:
terms = list(set.union(*[set(next(zip(*x.factor()))) for x in P_monoms - P_invol_monoms if x != 0]))

In [29]:
terms

[x_1^2*x_2^2 + x_1*x_2*x_3*x_4 + x_3^2*x_4^2,
 x_3,
 x_2,
 x_1,
 x_4,
 x_1^2*x_2^2 - x_1*x_2*x_3*x_4 + x_3^2*x_4^2,
 -x_1*x_2 + x_3*x_4,
 x_6,
 x_1^4*x_2^4 + x_1^3*x_2^3*x_3*x_4 + x_1^2*x_2^2*x_3^2*x_4^2 + x_1*x_2*x_3^3*x_4^3 + x_3^4*x_4^4,
 x_5,
 x_1^2*x_2^2 + x_3^2*x_4^2,
 x_0,
 x_1*x_2 + x_3*x_4]

In [38]:
[(x / terms[12]).factor() for x in P_monoms - P_invol_monoms if x != 0]

[(-1) * x_3^4 * x_1^4 * (-x_1*x_2 + x_3*x_4),
 x_3^4 * x_1^4 * (-x_1*x_2 + x_3*x_4),
 (-1) * x_0 * x_3^3 * x_1^3 * (x_1*x_2 + x_3*x_4)^-1 * (-x_1*x_2 + x_3*x_4) * (x_1^2*x_2^2 + x_1*x_2*x_3*x_4 + x_3^2*x_4^2),
 (-1) * x_4 * x_2 * x_0 * x_3^4 * x_1^4 * (x_1*x_2 + x_3*x_4)^-1 * (-x_1*x_2 + x_3*x_4),
 (-1) * x_5 * x_3^3 * x_1^3 * (-x_1*x_2 + x_3*x_4),
 x_4 * x_2 * x_0 * x_3^4 * x_1^4 * (x_1*x_2 + x_3*x_4)^-1 * (-x_1*x_2 + x_3*x_4),
 x_0 * x_3^3 * x_1^3 * (x_1*x_2 + x_3*x_4)^-1 * (-x_1*x_2 + x_3*x_4) * (x_1^2*x_2^2 + x_1*x_2*x_3*x_4 + x_3^2*x_4^2),
 x_5 * x_3^3 * x_1^3 * (-x_1*x_2 + x_3*x_4),
 (-1) * x_3^2 * x_1^2 * x_0^2 * (-x_1*x_2 + x_3*x_4) * (x_1^2*x_2^2 + x_3^2*x_4^2),
 (-1) * x_4 * x_2 * x_0^2 * x_3^3 * x_1^3 * (-x_1*x_2 + x_3*x_4),
 (-1) * x_5 * x_0 * x_3^2 * x_1^2 * (x_1*x_2 + x_3*x_4)^-1 * (-x_1*x_2 + x_3*x_4) * (x_1^2*x_2^2 + x_1*x_2*x_3*x_4 + x_3^2*x_4^2),
 (-1) * x_5 * x_4 * x_2 * x_0 * x_3^3 * x_1^3 * (x_1*x_2 + x_3*x_4)^-1 * (-x_1*x_2 + x_3*x_4),
 (-1) * x_6 * x_3^2 * x_1^2 

In [ ]:
a = list(product(*[set(next(zip(*x.factor()))) for x in P_monoms - P_invol_monoms if x != 0]))

In [436]:
a

In [431]:
set.union(*[set(next(zip(*x.factor()))) for x in P_monoms - P_invol_monoms if x != 0])

{x_6,
 x_5,
 x_4,
 x_3,
 x_2,
 x_1,
 x_0,
 -x_1*x_2 + x_3*x_4,
 x_1*x_2 + x_3*x_4,
 x_1^2*x_2^2 + x_3^2*x_4^2,
 x_1^2*x_2^2 - x_1*x_2*x_3*x_4 + x_3^2*x_4^2,
 x_1^2*x_2^2 + x_1*x_2*x_3*x_4 + x_3^2*x_4^2,
 x_1^4*x_2^4 + x_1^3*x_2^3*x_3*x_4 + x_1^2*x_2^2*x_3^2*x_4^2 + x_1*x_2*x_3^3*x_4^3 + x_3^4*x_4^4}

In [302]:
P_monoms = np.array([x.reduce(srideal) for x in P_monoms])
P_invol_monoms = np.array([x.reduce(srideal) for x in P_invol_monoms])

In [412]:
[x.reduce(srideal) for x in P_monoms - P_invol_monoms]

[x_1^6*x_2^2*x_3^4,
 0,
 -x_1^6*x_2^2*x_3^4,
 x_0*x_1^6*x_2^3*x_3^3,
 0,
 x_1^5*x_2^2*x_3^3*x_5,
 0,
 0,
 -x_0*x_1^6*x_2^3*x_3^3,
 -x_1^5*x_2^2*x_3^3*x_5,
 x_0^2*x_1^6*x_2^4*x_3^2,
 0,
 0,
 0,
 0,
 0,
 x_1^4*x_2^2*x_3^2*x_5^2,
 0,
 0,
 0,
 0,
 -x_0^2*x_1^6*x_2^4*x_3^2,
 0,
 -x_1^4*x_2^2*x_3^2*x_5^2,
 x_0^3*x_1^6*x_2^5*x_3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 x_1^3*x_2^2*x_3*x_5^3,
 0,
 0,
 0,
 0,
 0,
 0,
 -x_0^3*x_1^6*x_2^5*x_3,
 0,
 0,
 -x_1^3*x_2^2*x_3*x_5^3,
 x_0^4*x_1^6*x_2^6 - x_0^4*x_3^6*x_4^6,
 0,
 0,
 0,
 0,
 x_0^2*x_1^3*x_2^3*x_6 - x_0^2*x_3^3*x_4^3*x_6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 x_1*x_2*x_5^2*x_6 - x_3*x_4*x_5^2*x_6,
 x_1^2*x_2^2*x_5^4 - x_3^2*x_4^2*x_5^4,
 0,
 0,
 -x_0^2*x_1^3*x_2^3*x_6 + x_0^2*x_3^3*x_4^3*x_6,
 0,
 0,
 0,
 -x_1*x_2*x_5^2*x_6 + x_3*x_4*x_5^2*x_6,
 0,
 -x_0^4*x_1^6*x_2^6 + x_0^4*x_3^6*x_4^6,
 0,
 0,
 0,
 -x_1^2*x_2^2*x_5^4 + x_3^2*x_4^2*x_5^4,
 -x_0^5*x_2*x_3^5*x_4^6,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [310]:
a = np.unique(P_monoms - P_invol_monoms)
b = a[1]

In [311]:
b.factor()

x_6 * x_1^3 * (-x_2^2*x_3 + x_4^2*x_5)

In [255]:
P_invol

x_6^2 + x_0^4*x_2*x_4*x_6 + x_0^3*x_1*x_2*x_4*x_6 + x_0^3*x_2^2*x_3*x_6 + x_0^3*x_4^2*x_5*x_6 + x_0^2*x_1^2*x_2*x_4*x_6 + x_0^2*x_1*x_2^2*x_3*x_6 + x_0^2*x_1*x_4^2*x_5*x_6 + x_0^2*x_2*x_3*x_4*x_5*x_6 + x_0*x_1^3*x_2*x_4*x_6 + x_0*x_1^2*x_2^2*x_3*x_6 + x_0*x_1^2*x_4^2*x_5*x_6 + x_0*x_1*x_2*x_3*x_4*x_5*x_6 + x_0*x_2^2*x_3^2*x_5*x_6 + x_0*x_3*x_4^2*x_5^2*x_6 + x_1^4*x_2*x_4*x_6 + x_1^3*x_2^2*x_3*x_6 + x_1^3*x_4^2*x_5*x_6 + x_1^2*x_2*x_3*x_4*x_5*x_6 + x_1*x_2^2*x_3^2*x_5*x_6 + x_1*x_3*x_4^2*x_5^2*x_6 + x_2*x_3^2*x_4*x_5^2*x_6 + x_0^8*x_2^2*x_4^2 + x_0^7*x_1*x_2^2*x_4^2 + x_0^7*x_2^3*x_3*x_4 + x_0^7*x_2*x_4^3*x_5 + x_0^6*x_1^2*x_2^2*x_4^2 + x_0^6*x_1*x_2^3*x_3*x_4 + x_0^6*x_1*x_2*x_4^3*x_5 + x_0^6*x_2^4*x_3^2 + x_0^6*x_2^2*x_3*x_4^2*x_5 + x_0^6*x_4^4*x_5^2 + x_0^5*x_1^3*x_2^2*x_4^2 + x_0^5*x_1^2*x_2^3*x_3*x_4 + x_0^5*x_1^2*x_2*x_4^3*x_5 + x_0^5*x_1*x_2^4*x_3^2 + x_0^5*x_1*x_2^2*x_3*x_4^2*x_5 + x_0^5*x_1*x_4^4*x_5^2 + x_0^5*x_2^3*x_3^2*x_4*x_5 + x_0^5*x_2*x_3*x_4^3*x_5^2 + x_0^4*x_1^4*x_2^2*

In [249]:
cohom_ring_gens_invol

(x_0, x_1, x_4, x_5, x_2, x_3, x_6)

In [250]:
cohom_ring.gens()

(x_0, x_1, x_2, x_3, x_4, x_5, x_6)

In [6]:
from sympy import symarray
xs = symarray('x', ndivs)
sxs = xs[[i ^ invol for i in range(len(xs))]]
((rescws.T @ sxs).prod() / (rescws.T @ xs).prod()).simplify()

(x_0 + x_1 + 2*x_3 + x_4 + 5*x_6)/(x_0 + x_1 + x_2 + 2*x_5 + 5*x_6)

In [7]:
(rescws.T @ sxs).prod()

(x_2 + x_4 + 2*x_6)*(x_0 + x_1 + x_3 + x_5 + 4*x_6)*(x_0 + x_1 + 2*x_3 + x_4 + 5*x_6)

In [8]:
(rescws.T @ xs).prod()

(x_2 + x_4 + 2*x_6)*(x_0 + x_1 + x_2 + 2*x_5 + 5*x_6)*(x_0 + x_1 + x_3 + x_5 + 4*x_6)

In [93]:
ex['OPLANES']

[{'OIDEAL': ['x3^2*x4-x5^2*x6'], 'ODIM': 7},
 {'OIDEAL': ['x1', 'x2', 'x3^2*x4+x5^2*x6'], 'ODIM': 3}]

In [94]:
import numpy as np
from sage.rings.integer_ring import ZZ
from sage.matrix.all import matrix
R = matrix(ZZ, rescws)
D = R[invol.array_form] - R
a = D.left_kernel().matrix()

In [83]:
from sage.geometry.polyhedron.constructor import Polyhedron
K = R.left_kernel()
K_poly = Polyhedron(lines=K.matrix().rows())
K_poly_pos = K_poly.intersection(Polyhedron(rays=np.eye(K.degree())))
K_pos = matrix(ZZ, K_poly_pos.rays_list())

In [95]:
a

[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 1 0 1 0 0]
[0 0 0 1 0 1 0]
[0 0 0 0 2 1 0]
[0 0 0 0 0 0 1]

In [92]:
a * R

[ 0  0 -1]
[ 0  0 -1]
[ 0  0  4]
[ 0  0 -2]
[ 0  0  3]

In [76]:
(a * R).row_space()

Free module of degree 3 and rank 2 over Integer Ring
Echelon basis matrix:
[2 0 1]
[0 1 1]

In [358]:
from sage.rings.rational_field import QQ
P = QQ[[f'x_{i}' for i in range(ndivs)]]
xs = matrix(P, P.gens())

In [374]:
a.row_space().matrix()

[1 0 0 0 0 0 0]
[0 1 0 0 0 0 0]
[0 0 1 0 1 0 0]
[0 0 0 1 0 1 0]
[0 0 0 0 2 1 0]
[0 0 0 0 0 0 1]

In [57]:
(a[:, invol.array_form] + a).row_space().matrix()

[2 0 0 0 0 0 0]
[0 2 0 0 0 0 0]
[0 0 2 0 2 0 0]
[0 0 0 1 0 1 0]
[0 0 0 0 0 0 2]

In [58]:
(a[:, invol.array_form] - a).row_space().matrix()

[ 0  0  2  1 -2 -1  0]

[{'OIDEAL': ['x3^2*x4-x5^2*x6'], 'ODIM': 7},
 {'OIDEAL': ['x1', 'x2', 'x3^2*x4+x5^2*x6'], 'ODIM': 3}]

In [164]:
ex['INVOL']

'{D3->D5,D5->D3,D4->D6,D6->D4}'

In [80]:
import scipy.linalg

scipy.linalg.null_space(D)

array([[ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.63245553,  0.31622777, -0.63245553,
         0.        ],
       [ 0.        ,  0.        ,  0.69610123, -0.15194939,  0.30389877,
         0.        ],
       [ 0.        ,  0.        , -0.15194939,  0.92402531,  0.15194939,
         0.        ],
       [ 0.        ,  0.        ,  0.30389877,  0.15194939,  0.69610123,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         1.        ]])

In [87]:
D = (rescws[invol.array_form] - rescws).T
U, S, V = np.linalg.svd(D)

In [88]:
np.diag(S)

array([[3.16227766, 0.        , 0.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [86]:
V

array([[ 0.        ,  0.        , -0.31622777,  0.63245553,  0.31622777,
        -0.63245553,  0.        ],
       [ 0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.63245553,  0.69610123, -0.15194939,
         0.30389877,  0.        ],
       [ 0.        ,  0.        ,  0.31622777, -0.15194939,  0.92402531,
         0.15194939,  0.        ],
       [ 0.        ,  0.        , -0.63245553,  0.30389877,  0.15194939,
         0.69610123,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ]])

In [70]:
U.shape

(7, 7)

In [72]:
np.diag(S) @ V

array([[0.        , 0.        , 3.16227766],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        ]])

In [ ]:
a.associated_primes()